In [1]:
#!/usr/bin/env python3

import pickle
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axs
from prettytable import PrettyTable
from operator import add
from matplotlib.patches import Patch
from matplotlib.lines import Line2D


from dataTy import dataTy
from dataTy import Output
from dataTy import ResultHelper

# Setting colors
Colors={}

blue = (0.2980392156862745, 0.4470588235294118, 0.6901960784313725, 1.0)
green = (0.3333333333333333, 0.6588235294117647, 0.40784313725490196, 1.0)
red= (0.7686274509803922, 0.3058823529411765, 0.3215686274509804, 1.0)
purple =(0.5058823529411764, 0.4470588235294118, 0.6980392156862745, 1.0)
yellow = (0.8, 0.7254901960784313, 0.4549019607843137, 1.0)
cyan = (0.39215686274509803, 0.7098039215686275, 0.803921568627451, 1.0)
Colors["omp-offload"] = blue
Colors["refactor"] = green
#"omp-offload", "omp-mask-at", "omp-mask-nofl"]
Colors["omp-mask-at"] = red
Colors["omp-mask-nofl"] = purple
Colors["omp-offset-at"] = yellow
Colors["omp-offset-at-2"] = cyan
Colors["omp-offset-at-cm"] = cyan
Colors["dce-at"] = purple
Colors["omp-uvm"] = 'k'
Colors["omp-cpu1"] = 'k'
# metrics colors
Colors["OMPRuntime"] = blue
Colors["Kernel"] = green
Colors["H2DTransfer"] = red
Colors["D2HTransfer"] = purple
Colors["UpdatePtr"] = yellow
Colors["Other"] = 'k'
# 'black', 'red', 'green', 'blue', 'cyan'

def geo_mean(iterable):
    a = np.array([ x for x in iterable if x != 0])
    if len(a) == 0:
        return 0
    return a.prod()**(1.0/len(a))
def Round(F):
    return round(F,3)
def getFirstItem(d):
    return next(iter(next(iter(d.values())).values()))
def open_pickle(default_name):
    pfile = default_name
    # Use this in terminal py
    #if len(sys.argv) > 1:
        # first arg is filename
    #    pfile = sys.argv[1]
    print("Opening " + pfile)
    with open(pfile,'rb') as f:
        result = pickle.load(f)
    return result
def load_result(pickle_file, auto_prefixdir = True):
    if auto_prefixdir == True:
        pickle_file = "./results/" + pickle_file
    result = open_pickle(pickle_file)
   # print(type(result))
    ResultHelper.preprocessing(result)
    #result.pickle_path = pickle_file
    return result
def store_result(pickle_file, result, auto_prefixdir = True):
    if auto_prefixdir == True:
        pickle_file = "./results/" + pickle_file
    with open(pickle_file, "wb") as f:
        result.pickle_path = pickle_file
        pickle.dump(result, f)
def PrettyTableGen(lists):
    PT = PrettyTable()
    PT.field_names = lists[0]
    for li in lists[1:]:
        PT.add_row(li)
    return PT

def PrettyTablePrint(name, func, result):
    lists = []
    lists.append([name] + [ config for config in ResultHelper.getConfigs(result)])
    for proj in ResultHelper.getProjs(result):
        row = [proj]
        for config in ResultHelper.getConfigs(result):
            row.append(func(result[config].get(proj, Output())))
        lists.append(row)
    print(PrettyTableGen(lists))
def Summarizer(result):
    PrettyTablePrint("Overview", lambda output : Round(output.time), result)

# To be replaced by other table printer
class Printer:
    def __init__(self, result, the_config = None, the_proj = None):
        if the_config == None :
             configs = result.keys()
        else:
            configs = [the_config]
        for config in configs:
            output_of_proj = result[config]
            if the_proj == None:
                projs = output_of_proj.keys()
            else:
                projs = [the_proj]
                    
            for proj in projs:
                # Print with PrettyTable
                output = output_of_proj[proj]
                PT = PrettyTable()
                T = ResultHelper.getErrorOrAvgTime(output)
                PT.title = "" + config + " " + proj + " " + str(Round(T))
                PT.field_names = ["Attr", "Time", "Count"]
                PT.align["Attr"] = "l"
                PT.add_row(["Total", Round(output.prof_time), ""])
                for name in output.prof_data:
                    d = output.prof_data[name]
                    PT.add_row([name, Round(d.Value), d.Count])
                print(PT)
class config:
    width = 1
    margin = 1
    left_margin = 1
    save_img = True

class PlotPrinter:
    def submit(name, save = False):
        # Make layout clear
        plt.tight_layout()
        
        if save == True or config.save_img:
                #plt.savefig("plot/" + name + ".png", format='png',dpi=300, edgecolor='k')
            path = "plot/" + name + ".eps"
            plt.savefig(path, format='eps', dpi=300, edgecolor='k')
            path = "plot/" + name + ".png"
            plt.savefig(path, format='png', dpi=300, edgecolor='k')
            print("Saved plot to " + path)
        plt.show()
        plt.clf()
        plt.rcParams["font.family"] = "Times New Roman"
        

class ColumnChartPrinter:
    def plot(result, norm=False):
        # Get projs
        projs = ResultHelper.getProjs(result)
        configs = ResultHelper.getConfigs(result)
        proj_count = len(projs)
        config_count = len(configs)
        if ResultHelper.isInvalid(result) :
            return
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        last_col = xpos_base[-1]
        
        if norm == True:
            factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Times"], norm)
        else:
            factors = {p:1 for p in projs}
        for c in configs:
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Times",dataTy()).Value * factors[p]
                height.append(time)
            plt.bar(xpos, height, width=config.width, label=c)
            last_col = xpos[-1]
            n += 1
        if norm == True:
            plt.ylabel('Normalized Execution Time (%)', fontweight='bold')
        else:
            plt.ylabel('Execution Time(sec)', fontweight='bold')
        
        plt.xlabel('Benchmarks', fontweight='bold')
        plt.title('Overview', fontweight='bold')
        # Insert in the center
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        plt.xticks(xticks, projs)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)
        plt.xlim(xpos_base[0] - config.left_margin, last_col + config.margin)
        PlotPrinter.submit("all")

class KernelTimePrinter:
    def plotStackChar(result, configs, proj, ind, metrics):
        N = len(configs)
        btn = [0] * N
        plist = []
        for m in metrics:
            time = []
            for c in configs:
                time.append(float(result[c][proj].prof_data[m].Value))
            p = plt.bar(ind, time, config.width, bottom=btn)
            plist.append(p)
            # Increase btn
            btn = list(map(add, btn, time))
        return plist

    def plot(result, proj, norm=False):
        configs = ResultHelper.getConfigs(result)
        metrics = ["Kernel"]
        #metrics = ["H2DTransfer", "D2HTransfer", "UpdatePtr"]
        #metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        config_count = len(configs)
        ind = np.arange(0, config_count * (config.width + 1),  config.width + 1)
        #ind = np.arange(N)    # the x locations for the groups

        plist = StackChartPrinter.plotStackChar(result, configs, proj, ind, metrics)

        plt.ylabel('Execution Time(sec)', fontweight='bold')
        plt.xlabel('Configs', fontweight='bold')
        plt.title('Data movement [' + proj + ']', fontweight='bold')
        plt.xticks(ind, [config.rename_config(x) for x in configs]) #plt.yticks(np.arange(0, 81, 10))
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3)
        plt.xlim(ind[0]- config.left_margin, ind[-1] + config.margin)
        PlotPrinter.submit(proj)
            
class StackChartPrinter:
    def plotStackChar(result, configs, proj, ind, metrics):
        N = len(configs)
        btn = [0] * N
        plist = []
        for m in metrics:
            time = []
            for c in configs:
                time.append(float(result[c][proj].prof_data[m].Value))
            p = plt.bar(ind, time, config.width, bottom=btn)
            plist.append(p)
            # Increase btn
            btn = list(map(add, btn, time))
        return plist

    def plot(result, proj, metrics=[], norm=False, title=""):
        configs = ResultHelper.getConfigs(result)
        if len(metrics) == 0:
            metrics = ["Other","OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        else:
            print(metrics)
        #metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        config_count = len(configs)
        ind = np.arange(0, config_count * (config.width + 1),  config.width + 1)
        #ind = np.arange(N)    # the x locations for the groups

        plist = StackChartPrinter.plotStackChar(result, configs, proj, ind, metrics)

        plt.ylabel('Execution Time(sec)', fontweight='bold')
        plt.xlabel('Configs', fontweight='bold')
        if len(title) > 0:
            plt.title(title + ' ' + proj, fontweight='bold')
        else:
            plt.title('[Breakdown] ' + proj, fontweight='bold')
        plt.xticks(ind, [config.rename_config(x) for x in configs]) #plt.yticks(np.arange(0, 81, 10))
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3)
        plt.xlim(ind[0]- config.left_margin, ind[-1] + config.margin)
        PlotPrinter.submit(proj)

class ATOptPrinter:
    def plot(results):
        for r in results:
            first = results[r]
            break
        opts = results.keys()
        
        projs = ResultHelper.getProjs(first)
        the_config = "omp-offload-at"
        #metrics = [ "Kernel"]
        metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        # Gen xpos and xticks
        xpos = []
        xticks = []
        ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        for proj in projs:
            xpos = xpos + ind
            xticks += [proj, proj+"-opt"]
            ind = [x + 2 * config.width + 2.5 * config.width for x in ind]

        btn = [0] * len(projs) * 2
        plist = []
        # calculate the factor
        factors = {}
        for proj in projs:
            sum = 0
            for m in metrics:
                sum += float(first[the_config][proj].prof_data[m].Value)
            factors[proj] = 100/sum
                
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                for o in opts:
                    time.append(f*float(results[o][the_config][proj].prof_data[m].Value))
                    
            plist.append(plt.bar(xpos, time, config.width, bottom=btn))
            btn = list(map(add, btn, time))
        plt.ylabel('Execution Time(%)', fontweight='bold')
        plt.title("AT Opt Comparison", fontweight='bold')
        # shift because of rotation
        plt.xticks([x - config.width/2 for x in xpos], xticks, rotation=40)
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        #plt.ylim(top=110)
        PlotPrinter.submit("ATOptEffect")
        
# Compare different at optimization
def at_opt():
    files = {}
    files["O0"]   = "./results/at-O0.p"
    files["Osm"]  = "./results/at-Osm.p"
    
    results = {}
    for f in files:
        results[f] = pickle.load(open(files[f], "rb"))
        print("Open "+ files[f])
        ResultHelper.preprocessing(results[f])
    ATOptPrinter.plot(results)

def rename_legend(metrics):
    ret = []
    for m in metrics:
        if m == "OMPRuntime":
            ret.append("Runtime")
            continue
        if m == "H2DTransfer":
            ret.append("H2D")
            continue
        if m == "D2HTransfer":
            ret.append("D2H")
            continue
        ret.append(m)
    return ret
def rename_config(config, scheme=False):
    if config == "omp-offload":
        return "Baseline"
    if config == "refactor":
        return "Flattened"
    if config == "dce-at":
        return "Table-based"
    if config == "omp-mask-at":
        if scheme == True:
            return "Mask-based"
        return "Deepcopy"
    if config == "omp-offset-at":
        if scheme == True:
            return "Index-based"
        return "Deepcopy"
    if config == "omp-offset-at-cm":
        if scheme == True:
            return "Index-based-opt"
        return "Deepcopy"
    if config == "omp-uvm":
        return "UVM"
    return config
def genLegendHandle(configs):
    handle = []
    ind = 1
    for c in configs:
        label = str(ind) + ": " + rename_config(c)
        handle.append(Line2D([0], [0], marker='<', color='w', label=label,
                          markerfacecolor='g', markersize=12))
        ind += 1
    return handle
def genMetricsHandle(metrics):
    handle = []
    for m in metrics[::-1]:
        #handle.append(Line2D([0], [0], lw=6, color=Colors[m], label=m))
        handle.append(Patch(facecolor=Colors[m],label=m))
    return handle
    
#plt.rcParams['figure.figsize'] = [15, 10]
#plt.rcParams['figure.dpi'] = 80
config.save_img = True
#plt.style.use('seaborn-deep') # choose style https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
#plt.style.use('default')
plt.style.use('seaborn-deep')
#plt.rcParams["font.family"] = "Times New Roman"
#plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [ ]:
def main():
    # Read from pickle
    #result = load_result("2mm.p")
    result = load_result("rodi.p")
    #result = load_result("2result.p")
    #result = load_result("result.p")
    #result = load_result("All-1.p")

    Summarizer(result)

    #ColumnChartPrinter.plot(result,True)
    #print(result["omp-mask-at"]["2mm"].stderrs)
    for p in ResultHelper.getProjs(result):
        #KernelTimePrinter.plot(result,p)
        kernel_only = ["Kernel"]
        data_movement_only = ["H2DTransfer", "D2HTransfer", "UpdatePtr"]
        withoutKernel = ["OMPRuntime","H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        #StackChartPrinter.plot(result, p, metrics=kernel_only, title="Kernel")
        #StackChartPrinter.plot(result, p, metrics=data_movement_only)
        #StackChartPrinter.plot(result, p, metrics=withoutKernel, title="Data Transfer")
        #StackChartPrinter.plot(result, p)
        pass
if __name__ == "__main__":
    main()

In [ ]:
# Used for dumping detailed data
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate

class dummyData:
    Value = 0
    
def TableGen(result, proj, configs):
    colLabels = ["[{0}]".format(proj)]
    rowLabelsDict = {}
    index = 0
    for config in configs:
        colLabels.append(config)
        output = result[config][proj].prof_data
        for attr in output:
            attr_count = attr + "_count"
            # init attr
            if rowLabelsDict.get(attr) == None:
                rowLabelsDict[attr] = [attr]
                rowLabelsDict[attr_count] = ["-"]
            CountDiff = index - len(rowLabelsDict[attr])
            for i in range(CountDiff):
                rowLabelsDict[attr].append("NaN")
                rowLabelsDict[attr_count].append("NaN")
            rowLabelsDict[attr].append(output[attr].Value)
            rowLabelsDict[attr_count].append(output[attr].Count)
        index = index + 1
    # Merge data to list
    clust_data = []
 
    for entry in rowLabelsDict:
        clust_data.append(rowLabelsDict[entry])
    
    #StackChartPrinter.plot(result, proj)
    display(HTML("<style>table, th, td {font-size: 15px}</style>" + tabulate.tabulate(clust_data, colLabels, tablefmt='html')))

def main():
    # Data
    #result = load_result("2result.p")
    result = load_result("rodi.p")
    #result = load_result("./results/All-1.p")
    #result = load_result("./results/polly.p")
    #result = open_pickle("./results/Result-June3-final.p")
    Summarizer(result)
    #PrettyTablePrint("Stdever", lambda output : output.stdev_time, result)
    PrettyTablePrint("EntryNum", lambda output : output.prof_data.get("ATTableSize",dummyData()).Value, result)
    #TableGen(result, "fdtd-apml")
    #TableGen(result, "doitgen", configs = ["omp-offload", "refactor", "dce-at"])
    #TableGen(result, "reg_detect", configs = ["omp-mask-at"])

main()

In [ ]:
# Get speedup data
def PrintFinalSpeedUp(result):
    baseline_config = "omp-offload"
    final_config = "omp-mask-at"
    projs = ResultHelper.getProjs(result)
    projs.remove('fdtd-apml')
    #projs = ["backprop", "myocyte", "pathfinder"]
    # ------------------------------------------------------------------------
    at_ratios = []
    lists = []
    lists.append(["Total"] + [ c for c in result])
    
    #for c in result:
    #    projs = result[c].keys()
    #    break
    for p in projs:
        
        baseline = result[baseline_config][p].time
        li = [p, 1]
        for c in result:
            if c == baseline_config:
                continue
            v = result[c][p].time
            ratio = v/  baseline
            li.append(Round(ratio))
            if c == final_config:
                at_ratios.append(ratio)
        lists.append(li)
    print(PrettyTableGen(lists))
    print(Round(geo_mean(at_ratios)))
    # ----------------------------------------------------------------------
    at_ratios = []
    lists = []
    lists.append(["Kernel"] + [ c for c in result])
    

    for p in projs:
        
        baseline = result[baseline_config][p].prof_data["Kernel"].Value
        li = [p, 1]
        for c in result:
            if c == baseline_config:
                continue
            v = result[c][p].prof_data["Kernel"].Value
            
            ratio = v/  baseline
            li.append(Round(ratio))
            if c == final_config:
                at_ratios.append(ratio)
        lists.append(li)
    print(PrettyTableGen(lists))
    print(Round(geo_mean(at_ratios)))
    # ----------------------------------------------------------------------
    at_ratios = []
    lists = []
    lists.append(["Data Movement"] + [ c for c in result])

    for p in projs:
        metrics = ["H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        baseline = 0
        for m in metrics:
            baseline += result[baseline_config][p].prof_data[m].Value
        li = [p, 1]
        for c in result:
            if c == baseline_config:
                continue
            v = 0
            for m in metrics:
                v += result[c][p].prof_data[m].Value
            ratio = v/  baseline
            li.append(Round(ratio))
            if c == final_config:
                at_ratios.append(ratio)
        lists.append(li)
    print(PrettyTableGen(lists))
    print(Round(geo_mean(at_ratios)))

def main():
    # Data
    result = load_result("rodi.p")
    #result = load_result("./results/polly.p")
    Summarizer(result)
    PrintFinalSpeedUp(result)
    
main()

In [ ]:
# Result merge
import os
# Note Merge is dangenous
# Merge result and save into r1, and r1 pickle_file if save_file is true
def mergeResult(master,slave, save_file = False):
    print("Merging two result")
    Summarizer(master)
    Summarizer(slave)
    #return
    for c in slave:
        #print(c)
        if master.get(c) == None:
            # Not exist config, insert
            master[c]= slave[c]
            #print("Found new config in slave")
            
        else:
            for p in slave[c]:
                master_output = master[c]
                if master_output.get(p) == None:
                    # Not exist proj, insert
                    master_output[p] = slave[c][p]
                    #print("Found new proj in slave")
                else:
                    # r1 is more high prior
                    pass
    if save_file == True:
        #print("Saving merged result to ./results/{0}".format(getFirstItem(r1).pickle_path))
        #store_result("rodi.p", master)
        pass
    Summarizer(master)
def main():
    #slave = load_result("2result.p")
    slave = load_result("rodi.p")
    master = load_result("result.p")

    mergeResult(master,slave, True)
main()

In [ ]:
# Modifier & Tester
r = load_result("result.p")
r["kernel-stats"] = r["omp-offset-at"]
#r["dce-at"] = r["omp-offload-at"]
r.pop('omp-offset-at', None)
#r.pop('omp-mask-nofl', None)
#Error in omp-offset-at-2      atax                 RE
#Error in omp-offset-at-2      fdtd-apml            RE
#Error in omp-mask-nofl        3mm                  RE
#Error in omp-mask-nofl        fdtd-apml            RE
Summarizer(r)


#store_result("result.p", r)

In [ ]:
# Ploting Final charts
class NativeOmpBreakDown:
    def plot(result, projs = []):
        if len(projs) == 0:
            projs = ResultHelper.getProjs(result)
        plt.figure(num=None, figsize=(5, 3), dpi=80, facecolor='w', edgecolor='k')
        if result.get('omp-offload') == None:
            print("omp-offload config not exist")
            return
        outputs = result['omp-offload']
        metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        xpos = []
        xticks = []
        #ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        xpos = (np.arange(0, len(projs)*config.width*2, config.width * 2 )) 
        for proj in projs:
            xticks.append(proj)
        print(xticks)
        
        btn = [0] * len(projs)
        # calculate the factor
        factors =  ResultHelper.getNormFactors(outputs, metrics)
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                time.append(f*float(outputs[proj].prof_data[m].Value))
                    
                    
            plt.bar(xpos, time, config.width, bottom=btn, color=Colors[m])
            btn = list(map(add, btn, time))
        #plt.ylabel('Execution Time(%)(log)', fontweight='bold')
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title("OpenMP Offloading Breakdown", fontweight='bold')
        plt.xticks([x for x in xpos], xticks)
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.05, 1.0),ncol=1)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        PlotPrinter.submit("NativeBreakDown", save = True)
class RefactorCompare:
    def plot(result, projs = []):
        if len(projs) == 0:
            projs = ResultHelper.getProjs(result)
        plt.figure(num=None, figsize=(7, 4), dpi=80, facecolor='w', edgecolor='k')
        outputs = result['omp-offload']
        refector = result['refactor']
        metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        xpos = []
        xticks = []
        #ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        xpos = (np.arange(0, len(projs)*config.width*2, config.width * 2 )) 
        for proj in projs:
            xticks.append(proj)
        print(xticks)
        
        # calculate the norm factor
        factors =  ResultHelper.getNormFactors(outputs, metrics)
        
        plist = []

        # origin
        btn = [0] * len(projs)
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                time.append(f*float(outputs[proj].prof_data[m].Value))
                #time.append(f*float(refector[proj].prof_data[m].Value))
            bars = plt.bar(xpos - config.width/3, time, config.width/2, color=Colors[m], bottom=btn)
            plist.append(bars)
            btn = list(map(add, btn, time))
        for bar in bars: # last stacks # insert texts
            height = bar.get_height()
            #plt.text(bar.get_x() + bar.get_width()/2.0, height + bar.get_y(), 'original', ha='center', va='bottom')
        # refactored
        btn = [0] * len(projs)
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                time.append(f*float(refector[proj].prof_data[m].Value))
            bars= plt.bar(xpos + config.width/3, time, config.width/2, color=Colors[m], bottom=btn)
            btn = list(map(add, btn, time))
        
        for bar in bars: # last stacks # insert texts
            height = bar.get_height()
            #plt.text(bar.get_x() + bar.get_width()/2.0, height + bar.get_y(), 'flattened', ha='center', va='bottom')
        
        for proj in projs:
            plt.vlines(xpos + config.width, 0, 120,linestyles = 'dotted')
        #plt.ylabel('Execution Time(%)(log)', fontweight='bold')
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title("Comparison with Manual Refactor", fontweight='bold')
        plt.xticks([x for x in xpos], xticks)
        #plt.legend(plist, metrics, bbox_to_anchor=(1.05, 1.0),ncol=1)
        #plt.legend(plist, rename_legend(metrics), bbox_to_anchor=(0.5, -0.1), ncol=5,handlelength=1.0)
        configs=["omp-offload", "refactor"]
        config_legend = plt.legend(handles=genLegendHandle(configs),bbox_to_anchor=(1.0, 0.5))
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.00, 1.0),ncol=1)
        plt.gca().add_artist(config_legend)
        
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        plt.ylim(0, 105)
        
        PlotPrinter.submit("RefactorCompare", save = True)

result = load_result("rodi.p")

NativeOmpBreakDown.plot(result, ["backprop", "myocyte", "pathfinder"])
RefactorCompare.plot(result, ["backprop", "myocyte", "pathfinder"])


In [ ]:
# Overview chart
class Overview:
    def plot(result):
        
        plt.figure(num=None, figsize=(9, 4), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        
        configs = ["omp-offload", "refactor", "omp-mask-at"]
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Times"], True)
        geomean_data = {}
        for c in configs:
            geomean_data[c] = []
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Times",dataTy()).Value * factors[p]
                height.append(time)
                geomean_data[c].append(time)
            plt.bar(xpos, height, width=config.width, color=Colors[c],label=rename_config(c))
            n += 1
        xpos = xpos[-1] + 2 * config.width
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors[c])
            xpos += 1 * config.width

        #return plt.clf()
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Overview', fontweight='bold')
         
        plt.xticks([x  for x in xpos_base], projs + ["geomean"], rotation=25)
        plt.legend([rename_config(c) for c in configs], bbox_to_anchor=(1.0, 1.0),ncol=1)
        plt.margins(x=0.01)
        plt.gca().get_xticklabels()[-1].set_color('red') 
        PlotPrinter.submit("Overview", save = True)
        
class OverviewBreakdown2:
    def plot(result):
        plt.figure(num=None, figsize=(13, 5), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        #configs = ["omp-offload", "refactor", "omp-mask-at"]
        #configs = ["omp-offload", "omp-mask-at","omp-offset-at-cm"]
        configs = ["omp-offload", "refactor", "omp-mask-at"]
        
        #configs = ["omp-offload", "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        metrics = ["Other","OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count * 1.3 + 1), config_count * 1.3 + 1)
        plt.vlines(xpos_base - config.width, 0, 180,linestyles = 'dotted')
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], metrics)
        for c in configs:
            outputs = result[c]
            # prepare data
            xpos = [x + (config.width * 1.3) * n for x in xpos_base]
            n += 1
            btn = [0] * len(projs)
            for m in metrics:
                time = []
                for proj in projs:
                    f = factors[proj]
                    #time.append(f*float(outputs[proj].prof_data[m].Value))
                    time.append(f*float(outputs.get(proj,Output()).prof_data.get(m,dataTy()).Value))
                plt.bar(xpos, time, config.width, color=Colors[m], bottom=btn, label=m)
                btn = list(map(add, btn, time))

        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Breakdown', fontweight='bold')
        plt.ylim(0, 105)
        # Insert in the center
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        #plt.xticks(xticks, projs)
        plt.xticks([x - config.width*2 for x in xticks], projs, rotation=25)
        config_legend = plt.legend(handles=genLegendHandle(configs),bbox_to_anchor=(1.0, 0.5))
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.00, 1.0),ncol=1)
        plt.gca().add_artist(config_legend)
        plt.margins(x=0.01)
        PlotPrinter.submit("BreakdownAll", save = True)
class KernelSlowdown:
    def plot(result):

        plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        configs = ["omp-offload", "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        #metrics = ["Kernel"]

        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Kernel"])
        geomean_data = {}
        for c in configs:
            geomean_data[c] = []
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Kernel",dataTy()).Value * factors[p]
                height.append(time)
                geomean_data[c].append(time)
            plt.bar(xpos, height, width=config.width, color=Colors[c],label=rename_config(c))
            n += 1
        xpos = xpos[-1] + 2 * config.width
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors[c])
            xpos += 1 * config.width

        #return plt.clf()
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Kernel Slowdown', fontweight='bold')
         
        plt.xticks([x  for x in xpos_base], projs + ["geomean"], rotation=25)
        #plt.legend([rename_config(c) for c in configs], bbox_to_anchor=(1.05, 1.0),ncol=1)
        plt.legend([rename_config(c, scheme=True) for c in configs], bbox_to_anchor=(1.0, 1.0),ncol=1)
        plt.margins(x=0.01)
        plt.gca().get_xticklabels()[-1].set_color('red')
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=1)
        
        # insert AT ratio
        n = 0
        for p in projs:
            output = result["kernel-stats"][p]
            geomean_data = []
            for stats in output.ir_stats:
                #print("{} {} {}".format(stats.AT_count,stats.fload_count,stats.inst_count))
                if stats.inst_count == 0:
                    continue
                val = float(stats.AT_count)/stats.inst_count
                #print(val)
                geomean_data.append(val)
            ratio = geo_mean(geomean_data)
            if len(geomean_data) == 0:
                ratio = 0.003
            string = "{}%".format(round(ratio*100,2))
            plt.text(xpos_base[n],10,string,bbox=props)
            n += 1
        
        PlotPrinter.submit("KernelSlowdown", save = True)

class DataTransfer:
    def plot(result):
        plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        
        configs = ["omp-offload", "refactor", "omp-mask-at"]
        metrics = [ "H2DTransfer", "D2HTransfer", "UpdatePtr","OMPRuntime"]
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count * 1.3 + 1), config_count * 1.3 + 1)
        geomean_data = {}

        factors =  ResultHelper.getNormFactors(result["omp-offload"], metrics)
        #factors =  ResultHelper.getNormFactors(result["refactor"], metrics)
        for c in configs:
            outputs = result[c]
            # prepare data
            xpos = [x + (config.width * 1.3) * n for x in xpos_base]
            n += 1
            btn = [0.0] * len(projs)
            for m in metrics:
                time = []
                for proj in projs:
                    f = factors[proj]
                    #time.append(f*float(outputs[proj].prof_data[m].Value))
                    t = float(outputs.get(proj,Output()).prof_data.get(m,dataTy()).Value)
                    time.append(f*t)
                plt.bar(xpos, time, config.width, color=Colors[m], bottom=btn, label=m)
                btn = list(map(add, btn, time))
                geomean_data[c] = btn
        xpos = xpos[-1] + config.width * 1.3 + 1
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors["Kernel"])
            xpos += 1.3 * config.width
        plt.vlines(xpos_base - config.width, 0, 180,linestyles = 'dotted')
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Overhead Reduction', fontweight='bold')
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        plt.xticks([x - config.width*2 for x in xticks], projs + ["geomean"], rotation=25)
        config_legend = plt.legend(handles=genLegendHandle(configs),bbox_to_anchor=(1.0, 0.5))
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.00, 1.0),ncol=1)
        plt.gca().add_artist(config_legend)
        plt.margins(x=0.01)
        plt.ylim(0,105)
        plt.gca().get_xticklabels()[-1].set_color('red') 
        PlotPrinter.submit("OriginalOverhead", save = True)
        
result = load_result("rodi.p")
#result = load_result("rodi.p")
#Overview.plot(result)
#OverviewBreakdown2.plot(result)
KernelSlowdown.plot(result)
#DataTransfer.plot(result)
# Optimizations
## mask-at, offset-at, offset-at w/o reuse
#print(result["refactor"]["doitgen"].prof_data["H2DTransfer"].Value)
#print(result["omp-offload"]["doitgen"].prof_data["H2DTransfer"].Value
output = result['kernel-stats']['backprop']
for stats in output.ir_stats:
    print("{} {} {}".format(stats.AT_count,stats.fload_count,stats.inst_count))

In [ ]:
# TODO Compare with UVM
# Breakdown between UVM and best / offset-cm or 
class UVMCompare:
    def plot(result, projs = []):
        if len(projs) == 0:
            projs = ResultHelper.getProjs(result)
        plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
        outputs = result['omp-mask-at']
        refector = result['omp-uvm']

        metrics = ["Other","OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer"]
        xpos = []
        xticks = []
        #ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        xpos = (np.arange(0, len(projs)*config.width*2, config.width * 2 )) 
        for proj in projs:
            xticks.append(proj)
        
        # calculate the norm factor
        factors =  ResultHelper.getNormFactors(refector, metrics)

        # origin
        btn = [0] * len(projs)
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                time.append(f*float(outputs[proj].prof_data[m].Value))
                #time.append(f*float(refector[proj].prof_data[m].Value))
            bars = plt.bar(xpos + config.width/3, time, config.width/2, color=Colors[m], bottom=btn)
            btn = list(map(add, btn, time))
        # refactored
        btn = [0] * len(projs)
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                time.append(f*float(refector.get(proj,Output()).prof_data.get(m,dataTy()).Value))
            bars= plt.bar(xpos - config.width/3, time, config.width/2, color=Colors[m], bottom=btn)
            btn = list(map(add, btn, time))

        
        for proj in projs:
            plt.vlines(xpos + config.width, 0, 180,linestyles = 'dotted')
        #plt.ylabel('Execution Time(%)(log)', fontweight='bold')
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title("Comparison with Manual Refactor", fontweight='bold')
        plt.xticks([x - 0.8 * config.width for x in xpos], xticks, rotation=25)     
        plt.ylim(0, 125)
        configs=["omp-uvm","omp-mask-at"]
        config_legend = plt.legend(handles=genLegendHandle(configs),bbox_to_anchor=(1.0, 0.5))
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.2, 1.0))
        plt.gca().add_artist(config_legend)
        plt.margins(x=0.01)
        PlotPrinter.submit("UVMCompare", save = True)

        
result = load_result("rodi.p")
#Summarizer(result)

UVMCompare.plot(result)

In [ ]:
class CPU1:
    def plot(result):
        
        plt.figure(num=None, figsize=(9, 4), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        
        configs = ["omp-offload", "refactor", "omp-mask-at",'omp-cpu1']
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Times"], True)
        geomean_data = {}
        for c in configs:
            geomean_data[c] = []
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Times",dataTy()).Value * factors[p]
                height.append(time)
                geomean_data[c].append(time)
            plt.bar(xpos, height, width=config.width, color=Colors[c],label=rename_config(c))
            n += 1
        xpos = xpos[-1] + 2 * config.width
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors[c])
            xpos += 1 * config.width

        #return plt.clf()
        plt.ylabel('Normalized execution time (%)(log)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Overview', fontweight='bold')
         
        plt.xticks([x  for x in xpos_base], projs + ["geomean"], rotation=25)
        plt.legend([rename_config(c) for c in configs], bbox_to_anchor=(1.0, 1.0),ncol=1)
        plt.margins(x=0.01)
        plt.gca().get_xticklabels()[-1].set_color('red') 
        plt.yscale("log")
        PlotPrinter.submit("CPU1Compare", save = True)
   

result = load_result("rodi.p")
CPU1.plot(result)

In [ ]:
class KernelBreakdown:
    def plot(result):
        plt.figure(num=None, figsize=(13, 5), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        #configs = ["omp-offload", "refactor", "omp-mask-at"]
        #configs = ["omp-offload", "omp-mask-at","omp-offset-at-cm"]
        #configs = ["omp-offload", "refactor", "omp-mask-at"]
        configs = [ "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        #configs = ["omp-offload", "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        metrics = ["Other","OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count * 1.3 + 1), config_count * 1.3 + 1)
        plt.vlines(xpos_base - config.width, 0, 180,linestyles = 'dotted')
        
        factors =  ResultHelper.getNormFactors(result["omp-offset-at"], metrics)
        for c in configs:
            outputs = result[c]
            # prepare data
            xpos = [x + (config.width * 1.3) * n for x in xpos_base]
            n += 1
            btn = [0] * len(projs)
            for m in metrics:
                time = []
                for proj in projs:
                    f = factors[proj]
                    #time.append(f*float(outputs[proj].prof_data[m].Value))
                    time.append(f*float(outputs.get(proj,Output()).prof_data.get(m,dataTy()).Value))
                plt.bar(xpos, time, config.width, color=Colors[m], bottom=btn, label=m)
                btn = list(map(add, btn, time))

        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Breakdown', fontweight='bold')
        plt.ylim(0, 105)
        # Insert in the center
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        #plt.xticks(xticks, projs)
        plt.xticks([x - config.width*2 for x in xticks], projs, rotation=25)
        #config_legend = plt.legend([rename_config(c, scheme=True) for c in configs],bbox_to_anchor=(1.0, 0.5))
        plt.legend(handles=genMetricsHandle(metrics), bbox_to_anchor=(1.00, 1.0),ncol=1)
        #plt.gca().add_artist(config_legend)
        plt.margins(x=0.01)
        PlotPrinter.submit("Kernel-Compare-breakdown", save = True)
result = load_result("rodi.p")
KernelBreakdown.plot(result)

In [ ]:
class KernelSlowdownPlusTable:
    def plot(result):

        plt.figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        configs = ["omp-offload", "dce-at", "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        #metrics = ["Kernel"]

        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Kernel"])
        geomean_data = {}
        for c in configs:
            geomean_data[c] = []
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Kernel",dataTy()).Value * factors[p] /100
                height.append(time)
                if p == 'kmeans' and 'c' == 'dce-at':
                    time = 1
                if p == 'kmeans' or p == 'backprop':
                    print(time)
                geomean_data[c].append(time)
            plt.bar(xpos, height, width=config.width, color=Colors[c],label=rename_config(c))
            n += 1
        xpos = xpos[-1] + 2 * config.width
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors[c])
            #print(geo_mean(geomean_data[c]))
            xpos += 1 * config.width

        #return plt.clf()
        
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Kernel Slowdown', fontweight='bold')
         
        plt.xticks([x  for x in xpos_base], projs + ["geomean"], rotation=25)
        #plt.legend([rename_config(c) for c in configs], bbox_to_anchor=(1.05, 1.0),ncol=1)
        plt.legend([rename_config(c, scheme=True) for c in configs], bbox_to_anchor=(1.0, 1.0),ncol=1)
        plt.margins(x=0.01)
        plt.gca().get_xticklabels()[-1].set_color('red')
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=1)
        

        plt.yscale("log")
        PlotPrinter.submit("KernelSlowdownPlusTable", save = True)
result = load_result("rodi.p")
KernelSlowdownPlusTable.plot(result)

In [ ]:
class Overview2:
    def plot(result):
        
        plt.figure(num=None, figsize=(9, 4), dpi=80, facecolor='w', edgecolor='k')
        # Get projs
        projs = ResultHelper.getProjs(result)
        
        configs = ["omp-offload", "dce-at", "omp-offset-at", "omp-offset-at-cm","omp-mask-at"]
        proj_count = len(projs)
        config_count = len(configs)
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        
        factors =  ResultHelper.getNormFactors(result["omp-offload"], ["Times"], True)
        geomean_data = {}
        for c in configs:
            geomean_data[c] = []
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c].get(p,Output()).prof_data.get("Times",dataTy()).Value * factors[p]
                height.append(time)
                geomean_data[c].append(time)
            plt.bar(xpos, height, width=config.width, color=Colors[c],label=rename_config(c))
            n += 1
        xpos = xpos[-1] + 2 * config.width
        xpos_base = np.append(xpos_base,xpos)
        for c in geomean_data:
            print(geo_mean(geomean_data[c]))
            plt.bar(xpos, geo_mean(geomean_data[c]), width=config.width, color=Colors[c])
            xpos += 1 * config.width

        #return plt.clf()
        plt.ylabel('Normalized execution time (%)', fontweight='bold')
        #plt.xlabel('Benchmarks', fontweight='bold')
        #plt.title('Overview', fontweight='bold')
         
        plt.xticks([x  for x in xpos_base], projs + ["geomean"], rotation=25)
        plt.legend([rename_config(c) for c in configs], bbox_to_anchor=(1.0, 1.0),ncol=1)
        plt.margins(x=0.01)
        plt.gca().get_xticklabels()[-1].set_color('red') 
        PlotPrinter.submit("Overview", save = True)
result = load_result("rodi.p")
Overview2.plot(result)